In [528]:
import numpy as np
from matplotlib import pyplot
from seaborn import load_dataset
from pandas import read_csv
from pandas import DataFrame
from math import log as ln

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression


In [529]:
df = load_dataset('titanic')

In [530]:
df.head(5)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [531]:
class data_reader:
    def preprocessor(df=df):
        ls=[]
        _ls_=[]
        df=df.drop(['deck'], axis=1)
        df=DataFrame(df.dropna(axis=0), index=range(len(df.dropna())))
        df=df.dropna(axis=0)
        df.index=range(len(df.index))
        df['survived']=[False if element==0 else True for element in df['survived']]
        df['sex']=[True if sex=='male' else False for sex in df['sex']]
        df=Column_cut(df=df, column='embarked')
        df=Column_cut(df=df,column='pclass')
        df=Column_cut(df=df,column='town')
        
        elem=df['who']=='man'
        elem=elem!=df['sex']
        [ls.append(tab) if type(
            tab)==int else _ls_.append(
            tab) for tab in [index if elem[index
                                          ]==True else ' ' for index in range(
                                                len(elem==True))]]

        df=df.drop(ls, axis=0)
        df=df.drop(['class', 'male', 'who', 'alive'], axis=1)
        df.index=range(len(df.index))

        return df

    def scanner_nan():
        return np.sum(np.sum(DataFrame([
       [  np.sum(df[column]==None),
          np.sum(df[column]=='nan'),
          np.sum(df[column]=='NaN'),
          np.sum(df[column]=='NAN') ]   for column in df.columns])))


In [532]:
df=data_reader.preprocessor(df)
data_reader.scanner_nan()

0

In [533]:
df.columns

Index(['Cherbourg', 'Queenstown', 'Southampton', '1.0', '2.0', '3.0', 'C', 'Q',
       'S', 'survived', 'sex', 'age', 'sibsp', 'parch', 'fare', 'alone'],
      dtype='object')

In [534]:
def Column_cut(df=df,column='column'):
    column_transformer = make_column_transformer(
                (OneHotEncoder(), [column]),remainder='passthrough')
    df=column_transformer.fit_transform(df)
    df = DataFrame(df, 
                columns=[name_generator.split(
                    '_')[-1] for name_generator in column_transformer.get_feature_names()])
    return df

In [535]:
df

,Cherbourg,Queenstown,Southampton,1.0,2.0,3.0,C,Q,S,survived,sex,age,sibsp,parch,fare,alone
0,0,0,1,0,0,1,0,0,1,False,True,22,1,0,7.25,False
1,1,0,0,1,0,0,1,0,0,True,False,38,1,0,71.2833,False
2,0,0,1,0,0,1,0,0,1,True,False,26,0,0,7.925,True
3,0,0,1,1,0,0,0,0,1,True,False,35,1,0,53.1,False
4,0,0,1,0,0,1,0,0,1,False,True,35,0,0,8.05,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532,0,0,1,0,1,0,0,0,1,False,True,39,0,0,26,True
533,0,0,1,0,1,0,0,0,1,True,False,45,0,0,13.5,True
534,0,0,1,1,0,0,0,0,1,True,True,42,0,0,26.2875,True
535,0,0,1,1,0,0,0,0,1,True,False,22,0,0,151.55,True


In [572]:
x,y=df.drop(['survived'], axis=1), df['survived']

In [573]:
x_train, x_test, y_train, y_test=train_test_split(x,y, test_size=0.2, shuffle=True)

In [574]:
model=LinearRegression().fit(x_train,y_train)

In [575]:
model.coef_, model.intercept_

(array([ 1.57735930e-02, -6.79042526e-03, -8.98316772e-03,  1.90615545e-01,
        -1.30642038e-02, -1.77551341e-01,  1.57735930e-02, -6.79042526e-03,
        -8.98316772e-03, -5.31117828e-01, -2.63238234e-03, -5.56840074e-02,
        -4.54631484e-02,  1.26942976e-04, -2.03510569e-02]),
 0.903453984237976)

In [576]:
predict=[True if value>0.5 else False for value in DataFrame(model.predict(x_test), index=x_test.index).T.values[0]]

In [578]:
np.sum(predict==DataFrame(y_test).T.values[0])/len(y_test)

0.8425925925925926

In [567]:
model.score(x_train, y_train)

0.4289008820258896

In [568]:
model.score(x_test, y_test)

0.31296487877260804